In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import datetime
import matplotlib.dates as mdates

In [2]:
from ipywidgets import interactive
import ipywidgets as widgets

In [3]:
def sigmoid(t):
    y = 1/ (1+np.exp(-1*t))
    return y

In [146]:
# A grid of time points (in days)
t = np.linspace(0, 700, 701)
def getQuarantine(dayQuarantine):
    day_all=0
    for x in dayQuarantine: 
        day_all = day_all + x
    for x in dateday:
            diff = (x-datetime.timedelta(day_all)-start_date).days
            dayQuarantine = dayQuarantine + (diff,)
            day_all = day_all+diff
    print(dayQuarantine)
    return dayQuarantine

def getSum(tx):
    s=0
    ty=()
    for x in tx:
        s=s+x
        ty = ty + (s,)
    return ty
def getContact(cx,tx,t,dt):
    s=0
    ty=getSum(tx)
    i=0
    y=0#fillnull(len(t))
    while i<len(cx)-1:
        y = y+(cx[i]-cx[i+1])*(1-sigmoid((t-ty[i+1])/dt[i+1]))
        i = i+1
    y=y+cx[i]
    return y
def fillnull(A, n):
    m = n-len(A)
    if m>0:
        A = np.concatenate([np.zeros(n-len(A)) , A])
    return A
def proceed_start_paramter():
    global last_date, reported_recovered,reported_death, reported_infected, dayQuarantine, reported_critically
    reported_recovered = getSum(reported_recovered)
    last_date = start_date + datetime.timedelta(days = (len(reported_infected)))
    print(last_date)
    length = abs(last_date - start_date)
    reported_recovered = fillnull(reported_recovered,length.days)
    reported_death = fillnull(reported_death,length.days)
    reported_infected = fillnull(reported_infected,length.days)
    reported_critically = fillnull(reported_critically,length.days)
    dayQuarantine = getQuarantine(dayQuarantine)
    

In [181]:
# The SIR model differential equations
def deriv(y, t, 
       dayQuarantine1,
       contact1,
       dayQuarantine2,
       contact2,
       dayQuarantine3,
       contact3 ,infection_probability,
      #probabilty to get well without strong symtoms. Meens, the person stays inisolated and from Ia changes to R in the time teta_sym. 
      asymtom_probability,
      #time for Incubation and time for symtoms
       teta_inc, teta_sym, 
        recovery_rate, recovery_rate_mild, death_rate,
        #natural_death
        delta,
        #natural_birth
        A,
         quarantine_probability=1):
        
    S, E, Ia, Iu, Is, R, Ra, D = y
    
    #sigmoid
    cx = contact[:(len(contact)-3)] + (contact1,contact2,contact3)
    tx = dayQuarantine[:(len(contact)-3)] + (dayQuarantine1,dayQuarantine2,dayQuarantine3)
    n = getContact(cx,tx,t,deltat + (1,1,1))
    
    #susceptible
    dSdt = -n*infection_probability*(Ia+Iu)/(S+E+Ia+Iu+R+Ra) * S + A - delta*S
    #Exposed
    dEdt = n*infection_probability*(Ia+Iu)/(S+E+Ia+Iu+R+Ra) * S - 1/teta_inc*E- delta*E 
    #Infected asymptomatic
    dIadt = (1/teta_inc * E - 1/(teta_sym - teta_inc)*Ia - delta*Ia 
             - quarantine_probability*1/(teta_sym - teta_inc)*Ia*(1-asymtom_probability)*n*infection_probability*(Ia+Iu)/(S+E+Ia+Iu+R+Ra)) 
    
   
    
    #Infected unregisterted (no quarantine)
    dIudt = 1/(teta_sym - teta_inc)*Ia*(asymtom_probability) - recovery_rate_mild * Iu - delta*Iu
    #Infected symptomatic (quarantine)
    dIsdt = 1/(teta_sym - teta_inc)*Ia*(1-asymtom_probability) - recovery_rate * Is - death_rate*Is
    
    #Recovered
    dRdt = recovery_rate * Is - delta*R 
    #Recovered without symptoms
    dRadt = recovery_rate_mild * Iu - delta*Ra 
    #Dead from Desease
    dDdt = death_rate * Is
    
    
    return dSdt, dEdt, dIadt,dIudt, dIsdt, dRdt, dRadt, dDdt

In [208]:
def solver( 
       dayQuarantine1,
       contact1,
       dayQuarantine2,
       contact2,
       dayQuarantine3,
       contact3):
    # Initial conditions vector
    y0 = S0, E0, Ia0, Iu0, Is0, R0, Ra0, D0
 
    #sigmoid
    #sigmoid
    cx = contact[:(len(contact)-3)] + (contact1,contact2,contact3)
    print(cx)
    tx = dayQuarantine[:(len(contact)-3)] + (dayQuarantine1,dayQuarantine2,dayQuarantine3)
    n = getContact(cx,tx,t,deltat + (1,1,1))
    
    # Integrate the SIR equations over the time grid, t.
    args=(
   dayQuarantine1,
   contact1,
   dayQuarantine2,
   contact2,
   dayQuarantine3,
   contact3 ,infection_probability_proz/100, asymptom_probability_proz/100, teta_inc, teta_sym, 
                                 1/recovery_day, 1/recovery_day_mild, death_rate_proz/100,
                                delta, A)
    ret = odeint(deriv, y0, t, args)
    S, E, Ia, Iu, Is, R, Ra, D = ret.T
    dates = [start_date + datetime.timedelta(xval) for xval in t]
    # Plot the data on three separate curves for S(t), E(t(, Ia(t), Is(t) and R(t)
    font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

    plt.rcParams.update({'font.size': 12})

    fig = plt.figure(facecolor='w',figsize=[20,30])

    ax0 = fig.add_subplot(313, axisbelow=True)    
    ax0.plot(dates, n, color='c', lw=2, label='Contacts Amount')
    ax0.set_xlabel('Time /days')
    ax0.set_ylabel('Contacts')
    ax0.grid(True)
    ax0.yaxis.set_tick_params(length=5)
    #big picture
    ax1 = fig.add_subplot(312, axisbelow=True)
    ax1.plot(dates, Iu, color='c', lw=2, label='Infected unregistered')
    ax1.plot(dates, Is, 'red', alpha=0.5, lw=2, label='Infected symtomatic')
    ax1.plot(dates, need_help_proz/100*Is, color='red', alpha=0.5, lw=4, ls="-", label='Need help')
    ax1.plot(dates, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
    ax1.plot(dates, Ra, 'g', alpha=0.5, lw=2, ls = ':', label='Recovered asymtomatic with immunity')
    ax1.plot(dates, D, 'k', alpha=0.5, lw=2, label='Dead')
    ax1.set_xlabel('Time /days')
    ax1.set_ylabel('Contacts')
    ax1.grid(True)
    ax1.yaxis.set_tick_params(length=5)
    ax1.set_ylim(0, max(Is)*1.5)
    
    #First 250 days
    ax2 = fig.add_subplot(311, axisbelow=True)
    ax2.plot(dates, Iu, color='c', lw=2, label='Infected unregistered')
    ax2.plot(dates, Is, 'red', alpha=0.5, lw=2, label='Infected symtomatic')
    ax2.plot(dates, need_help_proz/100*Is, color='red', alpha=0.5, lw=4, ls="-", label='Need help')
    ax2.plot(dates, R, 'g', alpha=0.5, lw=2, label='Recovered with immunity')
    ax2.plot(dates, Ra, 'g', alpha=0.5, lw=2, ls = ':', label='Recovered asymtomatic with immunity')
    ax2.plot(dates, D, 'k', alpha=0.5, lw=2, label='Dead')
    ax2.set_xlabel('Time /days')
    ax2.set_ylabel('Contacts')
    ax2.grid(True)
    ax2.set_ylim(0, bed_capacity*5)
    ax2.set_xlim(start_date,start_date +datetime.timedelta(250))
    ax2.yaxis.set_tick_params(length=5)
    #
    ax0.xaxis.set_major_locator(mdates.MonthLocator())
    ax0.xaxis.set_minor_locator(mdates.DayLocator(bymonthday=(1,5,10,15,20,25,30)))
    ax0.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))
    ax0.xaxis.set_tick_params(length=5)
    
    ax1.xaxis.set_major_locator(mdates.MonthLocator())
    ax1.xaxis.set_minor_locator(mdates.DayLocator(bymonthday=(1,5,10,15,20,25,30)))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))

    ax2.xaxis.set_major_locator(mdates.MonthLocator())
    ax2.xaxis.set_minor_locator(mdates.DayLocator(bymonthday=(1,5,10,15,20,25,30)))
    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%b %d"))
    
    day1 = start_date + datetime.timedelta(dayQuarantine1)
    day2 = day1 + datetime.timedelta(dayQuarantine2)
    day3 = day2 + datetime.timedelta(dayQuarantine3)
    i=0
    day = start_date
    preday = start_date
    #colors=plt.rainbow(np.linspace(0,1,len(tx)))
    colors = ("cornsilk","snow","azure")
    for tt in tx:    
        preday = day
       
        day = day +datetime.timedelta(tt)
        ax1.axvline(x=day, color = "k", lw=2, label="Q w. "+str(cx[i]) + "contacts on " + 
                   str(day), ls=":")
        ax2.axvline(x=day, color = "k", lw=2, label="Q w. "+str(cx[i]) + "contacts on " + 
                   str(day), ls=":")
        

        if i>0:
            plt.axvspan(preday, day, facecolor=colors[(i%3)-1], alpha=0.5)
        i=i+1

    
    ax1.axhline(y=bed_capacity, color = "k", lw=3, ls="-", label="Bed capacity vs. " + 
                str(round(max(Is)*need_help_proz/bed_capacity,0))+"%")
    ax2.axhline(y=bed_capacity, color = "k", lw="3", label="Bed capacity vs." + 
                str(round(max(Is)*need_help_proz/100,0)))
    
    #plot reported data
    ax2.plot(dates[:len(reported_death)],reported_death,'kx', label='Reported deaths')
    ax2.plot(dates[:len(reported_critically)],reported_critically,'rx', label='Reported critically')
    ax2.plot(dates[:len(reported_recovered)],reported_recovered,'go', label='Reported recovered')
    ax2.plot(dates[:len(reported_infected)],reported_infected,'ro', label='Reported infected')

    legend = ax1.legend()
    legend = ax2.legend()
    
    prediction=(datetime.date.today() - start_date).days+7
    dat = datetime.date.today() + datetime.timedelta(days = 7)
    dat_last = last_date + datetime.timedelta(days = len(D)-1)
    lastnown = len(reported_infected)-1
    
    print(str(last_date) + " Infected: " + str(round(Is[lastnown],0)) +
          "Diff: " + str(round(Is[lastnown],0) - reported_infected[lastnown]))
    print(str(last_date) + " Dead: " + str(round(D[lastnown],0)) + 
          "Diff: " + str(round(D[lastnown],0) - reported_death[lastnown]))
    print(str(dat) + " Infected: " + str(round(Is[prediction],0)) + " Dead: " + str(round(D[prediction],0)))
    print(str(dat_last) + " Prediction Dead: " + str(round(D[len(D)-1],0)))
    plt.show()

In [209]:
def show_plot():
    interactive_plot = interactive(solver, 
                               dayQuarantine1=dayQuarantine[len(dayQuarantine)-3],
                               contact1 =contact[len(dayQuarantine)-3],
                               dayQuarantine2=dayQuarantine[len(dayQuarantine)-2],
                               contact2 =contact[len(dayQuarantine)-2],
                               dayQuarantine3=dayQuarantine[len(dayQuarantine)-1],
                               contact3 =contact[len(dayQuarantine)-1]
                               
                               #infection_probability_proz=(0.500*infection_probability_proz, 1.500*infection_probability_proz,0.02),
                               #asymptom_probability_proz=(asymptom_probability_proz*0.5,asymptom_probability_proz*1.5, 1),
                               #teta_inc = (teta_inc*0.5, teta_inc*1.5,0.1),
                               #teta_sym = (teta_sym*0.5, teta_sym*1.5,0.1),
                               #recovery_day=(recovery_day*0.5, recovery_day*1.5,0.1),
                               #recovery_day_mild=(recovery_day_mild*0.5, recovery_day_mild*1.5,0.1),
                               #death_rate_proz = (round(death_rate_proz*0.5,1), round(death_rate_proz*1.5,1), 0.1),
                               #need_help_proz = (need_help_proz*0.5,need_help_proz*1.5,0.5),
                               #bed_capacity = (bed_capacity*0.5, bed_capacity*1.5, bed_capacity/10)
                              )
    output = interactive_plot.children[-1]
    output.layout.height = '1000px'
    return interactive_plot

In [210]:
##Germany

# Total population, N.
N = 80*1000000
#natural death rate
delta = 11/1000/365
#natural birth_rate https://www.destatis.de/EN/Themes/Society-Environment/Population/Births/Tables/birth-deaths.html;jsessionid=D12FBB60A572EE37276ED5A4E5B23FE5.internet711
A = 9.5/1000*N/365
# Initial number of exposed, infected and recovered individuals.
E0, Ia0, Is0, R0, Ra0, D0 = 500, 70, 3, 0, 2, 0
Iu0 = 0
start_date = datetime.date(2020,2,15)

reported_infected = (0,0,0,0,0,0,0,0,0,3,11,32,58,63,114,149,187,246,528,652,782,1022,1204,1545,1938,2714,3621,4544,5754,7188,9274,12194,15161,19600,22071,24513,28480,29542,33570,37998,43862,
                    48781,52683,52740,
                    54933,
                    58350,
                    61247,
                    65309,
                    68248,
                    69839,
                    72865,69566,64647,63221, 65522,65181,64532,62578,60515,58349,56646,53931,
                     53786,53100,50703,48167,45933)
reported_death = (3,6,8,9,13,17,26,28,44,68,84,94,123,159,206,262,351,433,541,645,775,
        931,1107,1275,1444,1584,2016,2349,2607, 2736,2871,3022,3194,3495,3804,4052,4352,4538,4642,4538,4642,4862,5086,
        5315         )
reported_recovered = (4, 2, 0, 4, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 
 7, 0, 21, 0, 0, 21, 0, 38, 10, 65, 29, 57, 187, 2837, 257, 2126, 
 985, 1823, 730, 4289,
2600,
2600,
3740,
1235,
1825,
2300,
0,7381,10219,6107,1506,3487,3900,4000,3900,4400,4400,6114,2286,2600,3500,3700,4200)

reported_critically = (0,)

#q1 : 16.03.2020 School closed
#q2 : 23.03.2020 People asked to stay Home
dayQuarantine=(0,30,7) 
contact=(14,6,1.1,2.5,2.8,5)
deltat = (0.1,1,0.1,0.1,1,1)

dateday=(
         datetime.date(2020,4,20),
         datetime.date(2020,5,20),
         datetime.date(2021,1,25))
proceed_start_paramter()

Gamma=0.042 #0.0433107
CurlyEpsilon= 0.0152658
Phi=  0.22073
Psi= 1.

bed_capacity = 28000
death_rate_proz = 0.26#CurlyEpsilon*100
need_help_proz = 5
 
infection_probability_proz=Gamma*100
asymptom_probability_proz=(1-Phi)*100
teta_inc = 5.1
teta_sym = 11.5
recovery_day= 11
recovery_day_mild= 7

# Everyone else, S0, is susceptible to infection initially.
S0 = N - (E0 + Ia0 + Is0 + R0 + D0 + Ra0)

show_plot()


2020-04-22
(0, 30, 7, 28, 30, 250)


interactive(children=(IntSlider(value=28, description='dayQuarantine1', max=84, min=-28), FloatSlider(value=2.…

In [192]:
##Italy

# Total population, N.
N = 60.48*1000000
#natural death rate
delta = 10.6/1000/365
#natural birth_rate https://www.macrotrends.net/countries/ITA/italy/birth-rate
A = 7.509/1000*N/365 
# Initial number of infected and recovered individuals, I0 and R0.
E0, Ia0, Is0, Iu0, R0, Ra0, D0 = 1, 0, 0, 0, 0, 0, 0
start_date = datetime.date(2020,1,31)
#start 15.02.2020

#Italy Data
reported_infected = (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19, 75, 152, 221, 310, 455, 593, 822, 1049, 1577, 1835, 2263, 2706, 3296, 3916, 5061, 6387, 
                     7985, 8514, 10590, 12839, 14955, 17705, 20603, 23073, 26062, 28710, 33190, 37860, 
                     42681, 46638, 50418, 54030, 57521, 62013, 66414, 70065, 73880, 75228,77635,80572,83049,85388,88274,
                     91246,93187,94067,95262,96877,98273,100269,102253,103616,104291,105418,106607,106962,107771,
                    108257,108237,107709,107699)
reported_death=(2, 3, 7, 11, 12, 17, 21, 29, 41, 52, 79, 107, 148, 197, 233, 366, 463, 631, 827, 1016, 1266, 
                1441, 1809, 2158, 2503, 2978, 3405, 4032, 4825, 5476, 6077, 6820, 7503, 8215, 9134, 10023, 10779,
                11591, 12428, 13155, 13915,14681,15362,15887,16523,17127,17669,18279,18849,19468,19899,20465,
                21067,21645,22170,22745,23227,23660,24114,24648,25085)

reported_recovered = (1,1,0,1,1,1,42,1,4,33,66,11,116,138,109,66,33,102,280,41,21,181,527,369,
414,192,1084,415,689,943,
952,408,894,1036,999,589,1434,646,1590,1109,1118,1431,1480,1238,819,1022,1555,2099,1979,1985,
2079,1677,1224,1695,962,2072,2563,2200,2128,1822,2723,2943)

reported_critically = (0,)

#q1 : 21.02.2020 in LOMBARDIE
#q2 : 09.03.2020 in Italy
dayQuarantine=(0,24.089,45.3509-24.089) 
dateday=(
         datetime.date(2020,3,30),
        datetime.date(2020,4,20),
         datetime.date(2020,5,20),
         datetime.date(2020,10,31))

contact=(35.1,7.36,1.9,1.5,3,1.5,3)
deltat = (1,4.97,1.572,1,1,1,1)

proceed_start_paramter()

Gamma=0.0433 #0.0433107
CurlyEpsilon= 0.011#0.0152658
Phi=  0.22073
Psi= 1.

bed_capacity = 28000
death_rate_proz = 0.9#CurlyEpsilon*100
need_help_proz = 5
 
infection_probability_proz=Gamma*100
asymptom_probability_proz=(1-Phi)*100
teta_inc = 5.1
teta_sym = 11.5
recovery_day= 5*7
recovery_day_mild= 7

# Everyone else, S0, is susceptible to infection initially.
S0 = N - (E0 + Ia0 + Is0 + R0 + D0 + Ra0)

show_plot()


2020-04-22
(0, 24.089, 21.261900000000004, 14, 21, 30, 164)


interactive(children=(IntSlider(value=21, description='dayQuarantine1', max=63, min=-21), IntSlider(value=3, d…

In [190]:
##Belarus

# Total population, N.
N = 9.485*1000000
#natural death rate
delta = 12/1000/365
#natural birth_rate https://www.destatis.de/EN/Themes/Society-Environment/Population/Births/Tables/birth-deaths.html;jsessionid=D12FBB60A572EE37276ED5A4E5B23FE5.internet711
A = 9.5/1000*N/365
# Initial number of exposed, infected and recovered individuals.
E0, Ia0, Is0, R0, Ra0, D0 = 5, 3, 0, 0, 0, 0
Iu0=0

start_date = datetime.date(2020,2,19)


reported_infected = (0,0,0,0,0,0,0,0,0,0,1,1,1,4,6,6,6,6,6,6,6,6,18,24,24,24,33,33,46,46,54,61,61,59,59,57,57,62,62,120,104,108,247,
                    294,382,502,634,794,976,1331,1793,2031,2349,2687,3045, 3489,3961,4395,
                     4770,5280,5798,
                    #4392,4238,5699,
                     6091,6454#7084
                    )
reported_death = (1,2,4,4,5,8,13,13,16,19,23,26,29,33,36,40,42,47,51,55,60)
reported_recovered = (33,458,494,514,769)

reported_critically = (92,92)



#q1 : 06.04.2020 School closed
dayQuarantine=(0,) 
contact=(11,7,6,7,9,9)
deltat = (1,1,1,1,1,1)

dateday=(
        datetime.date(2020,4,5), # 
        datetime.date(2020,4,10), #who in Minsk 13.04
        datetime.date(2020,4,20), #schule offen 
         datetime.date(2020,5,20),
         datetime.date(2020,10,31))
proceed_start_paramter()

Gamma=0.039 #0.0433107
CurlyEpsilon= 0.0152658
Phi=  0.22073
Psi= 1.

bed_capacity = 3000
death_rate_proz = 0.26#0.17#CurlyEpsilon*100
need_help_proz = 5
 
infection_probability_proz=Gamma*100
asymptom_probability_proz=(1-Phi)*100
teta_inc = 5.1
teta_sym = 11.5
recovery_day= 20
recovery_day_mild= 7

# Everyone else, S0, is susceptible to infection initially.
S0 = N - (E0 + Ia0 + Is0 + R0 + D0 + Ra0)

show_plot()



2020-04-22
(0, 46, 5, 10, 30, 164)


interactive(children=(IntSlider(value=10, description='dayQuarantine1', max=30, min=-10), IntSlider(value=7, d…

In [12]:
#Import Data
#import pandas as pd
#arr = pd.read_csv('infectedDEraw.dat', sep=';',quotechar='"', quoting=2,na_values = "\\N", error_bad_lines=False)
#s=""
#arr = arr.to_numpy()
#for xval in arr:
#    s = s + "," + str(int(xval[0]))
#print(s)
#print(arr)

